# convert CMG sr3 file to rwo files

sr3 is the CMG simulation result file, whereas rwo is the extracted info in the ascii format

In [ ]:
import os
from CMG2npy import generate_CMG_rwd, run_CMG_rwd_report

folder_path = "data"
for case_num in range(1, 3):
    generate_CMG_rwd(
        sr3_folder_path = folder_path,
        case_name = f'case{case_num}',
        property = 'GEORTYPE',
        is_gmc_property = True,
        precision = 4
    )

    run_CMG_rwd_report(
        rwd_folder_path = folder_path,
        case_name = f'case{case_num}',
        cmg_version = 'ese-ts2win-v2024.20',
    )

    # remove rwd files
    rwd_folder_path = folder_path
    os.remove(os.path.join(rwd_folder_path, f'case{case_num}.rwd'))

# extract simulation results from rwo files into numpy arrays

In [ ]:
from CMG2npy import CMG_rwo2npy

for case_num in range(1, 3):
    CMG_rwo2npy(
        rwo_folder_path = "data\\rwo",
        case_name = f'case{case_num}',
        property = 'PRES',
        save_folder_path = "results",
        show_info = False
    )

Parsing file: data/case1_PRES.rwo
Processing time step: 0.0 (2030-Jan-01)
Processing time step: 3652.0 (2040-Jan-01)
Processing time step: 7305.0 (2050-Jan-01)
Processing time step: 10957.0 (2060-Jan-01)
Processing time step: 189926.0 (2550-Jan-01)
Processing time step: 372547.0 (3050-Jan-01)
Found 6 time steps
Grid dimensions: I=107, J=117, K=79, Time=6


# extract simulation results of simulated layers

for the JD_Sula_2025_flow grid (107x117x79), the reservoir layers are k=41-79. The other layers were not simulated by setting ntg=0.

In [ ]:
import os
import numpy as np

# Input and output folders
input_folder = "data"             
output_folder = "results_trimmed"    
os.makedirs(output_folder, exist_ok=True)

# Loop through files
for filename in os.listdir(input_folder):
    if filename.endswith("_PRES.npy"):
        input_path = os.path.join(input_folder, filename)

        # Load array
        arr = np.load(input_path) 

        # Extract slices 41–79 (Python indices are 0-based, so use 40:79)
        arr_trimmed = arr[:, :, 40:79]   # shape (107,117,39,6)

        # Save to output folder with same name
        output_path = os.path.join(output_folder, filename)
        np.save(output_path, arr_trimmed)

        print(f"Processed {filename} -> {arr_trimmed.shape}")
